In [2]:
# ! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [3]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'

os.getenv('LANGCHAIN_TRACING_V2')
os.getenv('LANGCHAIN_ENDPOINT')
os.getenv('LANGCHAIN_API_KEY')
os.getenv('OPENAI_API_KEY')

'sk-proj-iXDvNFgNThedT3VorkIfT3BlbkFJkCBkiTpSBlaj3GBnm5v1'

In [4]:
# !pip install pymupdf

In [5]:
!ls

AddAutogenTools.ipynb
QueryTranslation.ipynb
QueryTranslationAndDocumentSelector.ipynb
README.md
SecInsights.ipynb
SecInsights.py
cfs.json
data
rag_from_scratch_5_to_9.ipynb
some.html
test.json


In [6]:
# from langchain.document_loaders import PyMuPDFLoader
# # loader = WebBaseLoader(
# #     web_paths=("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf",),
# #     bs_kwargs=dict(
# #         parse_only=bs4.SoupStrainer(
# #             class_=("post-content", "post-title", "post-header")
# #         )
# #     ),
# # )
# # blog_docs = loader.load()



# # Load PDF
# loader = PyMuPDFLoader("data/amazon_form_10k_2023.pdf")
# pdf_docs = loader.load()

# # You can now use pdf_docs for further processing
# for doc in pdf_docs:
#     print(doc.page_content)

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.document_loaders import PyMuPDFLoader
# loader = WebBaseLoader(
#     web_paths=("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# blog_docs = loader.load()



# Load PDF
loader = PyMuPDFLoader("data/amazon_form_10k_2023.pdf")
pdf_docs = loader.load()

# # You can now use pdf_docs for further processing
# for doc in pdf_docs:
#     print(doc.page_content)


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(pdf_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

# MultiQuery

In [8]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [9]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

c:\Users\goldr\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


6

In [10]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Some of the biggest discussed risks for Amazon.com Inc. (AMZN) include legal and regulatory risks related to government regulations, evolving laws, and potential lawsuits, as well as risks related to intense competition in the rapidly evolving and competitive business environment. Additionally, risks related to maintaining a unique culture of innovation, disruptions from natural disasters or geopolitical events, and potential negative impacts of climate change are also highlighted as significant risks for the company.'

# Rag Fusion

In [11]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [13]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

12

In [14]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'The biggest discussed risks for Amazon.com Inc. (AMZN) include maintaining their unique culture of innovation, customer obsession, and long-term thinking, disruptions from natural or human-caused disasters, geopolitical events, security issues, labor or trade disputes, potential negative impacts of climate change, challenges in staffing, developing, and managing foreign operations, compliance with various laws and regulations, cybersecurity risks, legal and regulatory risks, and variability in their retail business placing strain on operations.'

# Decomposition

In [15]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
questions = generate_queries_decomposition.invoke({"question":question})

In [17]:
questions

['1. What are the current regulatory risks facing Amazon.com Inc. (AMZN)?',
 '2. How does competition from other e-commerce giants impact the risk profile of Amazon.com Inc. (AMZN)?',
 "3. What are the potential cybersecurity threats that pose risks to Amazon.com Inc. (AMZN)'s operations and data security?"]

## sequential decomposition

In [18]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question: 

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [19]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""
    
    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

q_a_pairs = ""
for q in questions:
    
    rag_chain = (
    {"context": itemgetter("question") | retriever, 
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")} 
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [20]:
answer

"Potential cybersecurity threats that pose risks to Amazon.com Inc. (AMZN)'s operations and data security include data loss, theft, misuse, unauthorized access, security breaches, and vulnerabilities affecting their technology, products, and systems. These threats could expose Amazon and its customers to the risk of loss, disclosure, or misuse of confidential, sensitive, proprietary, and personal information. Additionally, such cybersecurity incidents could adversely affect Amazon's operating results, lead to litigation, liability, regulatory actions, deter customers or sellers from using their services, and harm their business reputation. Amazon uses third-party technology and systems for various functions, including encryption and authentication, which could introduce additional risks if not properly secured. The company conducts application security assessments, vulnerability management, penetration testing, security audits, and ongoing risk assessments to mitigate these cybersecuri

In [21]:
# Answer each sub-question individually 

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

c:\Users\goldr\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [22]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'The biggest discussed risks for Amazon.com Inc. (AMZN) include financial risks such as fluctuations in foreign exchange rates, changes in global economic conditions, and competition. Regulatory risks are also a significant concern, with evolving government regulations around taxation, privacy, data protection, and consumer protection posing potential threats to the company. Additionally, cybersecurity risks related to data loss, security breaches, and unauthorized access are key areas of focus for Amazon to manage and mitigate.'

# StepBack 

In [ ]:
# # Few Shot Examples
# from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
# examples = [
#     {
#         "input": "Could the members of The Police perform lawful arrests?",
#         "output": "what can the members of The Police do?",
#     },
#     {
#         "input": "Jan Sindel’s was born in what country?",
#         "output": "what is Jan Sindel’s personal history?",
#     },
# ]
# # We now transform these to example messages
# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("human", "input"),
#         ("ai", "output"),
#     ]
# )
# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
# )
# prompt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
#         ),
#         # Few shot examples
#         few_shot_prompt,
#         # New question
#         ("user", "{question}"),
#     ]
# )

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """You are an expert in world knowledge. Your task is to paraphrase a specific question into a more generic and broader question, making it easier to answer. Here are a few examples:"""
        ),
        # Few-shot examples
        ("user", 
            "Could the members of The Police perform lawful arrests?"
        ),
        ("assistant", 
            "What can the members of The Police do?"
        ),
        ("user", 
            "Jan Sindel was born in what country?"
        ),
        ("assistant", 
            "What is Jan Sindel’s personal history?"
        ),
        # New question
        ("user", "{question}"),
        ("assistant", "")
    ]
)


In [28]:
generate_queries_step_back = prompt | ChatOpenAI(temperature=0) | StrOutputParser()
question = "What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
generate_queries_step_back.invoke({"question": question})

'What are some of the key risks associated with investing in Amazon.com Inc. (AMZN)?'

In [ ]:
# Response prompt 
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

"Some of the biggest discussed risks for Amazon.com Inc. (AMZN) include:\n\n1. Maintaining a culture of innovation, customer obsession, and long-term thinking: Amazon's growth and success are heavily reliant on its unique culture, and any failure to maintain this culture could hinder its performance.\n\n2. Disruptions from natural or human-caused disasters: Events such as public health crises, extreme weather, geopolitical issues, and labor disputes can disrupt Amazon's operations and impact its financial performance.\n\n3. Negative impacts of climate change: Climate change can lead to increased operating costs, changes in customer behavior, compliance costs, and reputational damage for Amazon, affecting its bottom line and public perception.\n\n4. Optimizing and operating fulfillment network and data centers: Failures in predicting customer demand, consumer spending patterns, or optimizing operations can result in inefficiencies and impact Amazon's ability to meet customer expectation

# HyDE

In [ ]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run
question = "What are the biggest discussed risks for Amazon.com Inc. (AMZN)?"
generate_docs_for_retrieval.invoke({"question":question})

'One of the biggest discussed risks for Amazon.com Inc. (AMZN) is its heavy reliance on third-party sellers for a significant portion of its revenue. This dependence exposes the company to potential risks such as counterfeit products, regulatory issues, and reputational damage. Another major risk for Amazon is its vulnerability to changes in consumer preferences and technological advancements, which could impact its competitive position in the e-commerce market. Additionally, the company faces regulatory scrutiny and antitrust concerns due to its dominant market position and potential anti-competitive practices. Overall, these risks highlight the need for Amazon to continuously monitor and address potential threats to its business operations and financial performance.'

In [ ]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'creationDate': "D:20240202060436-05'00'", 'creator': 'EDGAR Filing HTML Converter', 'encryption': 'Standard V2 R3 128-bit RC4', 'file_path': 'data/amazon_form_10k_2023.pdf', 'format': 'PDF 1.4', 'keywords': '0001018724-24-000008; ; 10-K', 'modDate': "D:20240202060451-05'00'", 'page': 5, 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': 'data/amazon_form_10k_2023.pdf', 'subject': 'Form 10-K filed on 2024-02-02 for the period ending 2023-12-31', 'title': '0001018724-24-000008', 'total_pages': 94, 'trapped': ''}, page_content='events and consequences discussed in these risk factors could, in circumstances we may or may not be able to accurately predict, recognize, or control, have a\nmaterial adverse effect on our business, growth, reputation, prospects, financial condition, operating results (including components of our financial results),\ncash flows, liquidity, and stock price. These risk 

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'The biggest discussed risks for Amazon.com Inc. (AMZN) include intense competition in rapidly evolving industries, potential negative impacts of climate change, disruptions from natural or human-caused disasters, geopolitical events and security issues, and risks related to optimizing and operating their fulfillment network and data centers.'

In [1]:
!pip install datasets


  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/547.8 kB ? eta -:--:--
   -------- ------------------------------- 112.6/547.8 kB 2.2 MB/s eta 0:00:01
   ----------------------------------- ---- 481.3/547.8 kB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 547.8/547.8 kB 4.9 MB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.1 MB 9.6 MB/s eta 0:00:03
   - -------------------------------------- 0.9/25.1 MB 10.9 MB/s eta 0:00:03
   - -------------------------------------- 1.2/25.1 MB 11.2 MB/s eta 0:00:03
   --- ------------------------------------ 2.0/25.1 MB 11.4 MB/s eta 0:00:03
   ---- ----------------------------------- 2.6/25.1 MB 11.7 MB/s eta 0:00:02
   ---- ----------------------------------- 2.9/25.